In [8]:
import pandas as pd
import dwsemear
import datetime

import postgresemear as sql # mover os arquivos para a pasta principal

# GERACAO DOS DADOS

In [9]:
def substituicao_de_NaT_para_None(dados, coluna):
    novas_datas = []
    for i in dados.index: 
        if str(dados.loc[i,coluna]) == 'NaT':
            novas_datas.append(None)
        else:
            novas_datas.append(dados.loc[i,coluna])
    return novas_datas


In [44]:
dados = pd.read_excel('3040_06_2019.xlsx', encoding='latin-1', dtype=str) # demora uns 15min
dados = dwsemear.limpa_nome_das_colunas(dados)

In [45]:
dados['Data'] = pd.to_datetime(dados['Data'], format='%Y-%m-%d %H:%M:%S')
dados['DatInicio'] = pd.to_datetime(dados['DatInicio'], format='%Y-%m-%d %H:%M:%S')
dados['DatProxParcela'] = pd.to_datetime(dados['DatProxParcela'], format='%Y-%m-%d %H:%M:%S')
dados['DatVencimento'] = pd.to_datetime(dados['DatVencimento'], format='%Y-%m-%d %H:%M:%S')
dados['DatRenegociacao'] = pd.to_datetime(dados['DatRenegociacao'], format='%Y-%m-%d %H:%M:%S')
dados = dwsemear.coluna_cpf_para_inteiro(dados, 'CpfCgc')
dados['CodProduto'] = dados['CodProduto'].astype(int)
dados['Contrato'] = dados['Contrato'].astype(int)
dados.loc[:,'DiasAtraso'] = dados.loc[:,'DiasAtraso'].astype(int)
dados['QtdParAberta'] = dados['QtdParAberta'].astype(int)
dados['VlrSaldo'] = dados['VlrSaldo'].astype(float)
dados['VlrPDD'] = dados['VlrPDD'].astype(float)
dados['VlrOperacao'] = dados['VlrOperacao'].astype(float)
dados['VlrRenda'] = dados['VlrRenda'].astype(float)
dados.loc[:, 'VlrProxParcela'] = dados.loc[:, 'VlrProxParcela'].astype(float)
# RatFim = str

In [46]:
Data = substituicao_de_NaT_para_None(dados, 'Data')
DatInicio = substituicao_de_NaT_para_None(dados, 'DatInicio')
DatProxParcela = substituicao_de_NaT_para_None(dados, 'DatProxParcela')
DatVencimento = substituicao_de_NaT_para_None(dados, 'DatVencimento')
DatRenegociacao = substituicao_de_NaT_para_None(dados, 'DatRenegociacao')

# ORGANIZACAO DOS DADOS PARA CARGA NO POSTGRESQL

In [47]:
id_pdd = sql.query_max_coluna('id', 'pdd')
if id_pdd == None:
    id_pdd = -1

In [48]:
pdd_registros = []
for i in dados.index:
    id_pdd += 1
    pdd_registros.append((
        int(id_pdd),
        Data[i],
        DatInicio[i],
        DatProxParcela[i],
        DatVencimento[i],
        DatRenegociacao[i],
        int(dados.loc[i,'CpfCgc']),
        int(dados.loc[i,'CodProduto']),
        int(dados.loc[i,'Contrato']),
        int(dados.loc[i,'DiasAtraso']),
        int(dados.loc[i,'QtdParAberta']),
        float(dados.loc[i,'VlrSaldo']),
        float(dados.loc[i,'VlrPDD']),
        float(dados.loc[i,'VlrOperacao']),
        float(dados.loc[i,'VlrRenda']),
        float(dados.loc[i, 'VlrProxParcela']),
        dados.loc[i, 'RatFim']))


# DEFINICAO DA TABELA

In [ ]:
#delete_tabela = '''DROP TABLE pdd'''
#sql.execute_query(delete_tabela)

In [136]:
#crie_tabela = '''CREATE TABLE pdd
#            (ID BIGINT PRIMARY KEY NOT NULL,
#            DataDeReferencia timestamp with time zone,
#            DataDeOriginiacao timestamp with time zone,
#            DataDaProximaParcela timestamp with time zone,
#            DataDeVencimento timestamp with time zone,
#            DataDeRenegociacao timestamp with time zone,
#            CPF BIGINT NOT NULL,
#            CodigoDoProduto INT NOT NULL,
#            CodigoDoContrato INT NOT NULL,
#            DiasEmAtraso INT NOT NULL,
#            QuantidadeDeParcelasAbertas INT NOT NULL,
#            ValorSaldo REAL NOT NULL,
#            ValorPDD REAL NOT NULL,
#            ValorOperacao REAL NOT NULL,
#            RendaDoCliente REAL,
#            ValorDaProximaParcela REAL NOT NULL,
#            RatingContabilFim TEXT NOT NULL);'''
#sql.execute_query(crie_tabela)

# CARGA

In [49]:
query = """ INSERT INTO pdd
            (ID,
            DataDeReferencia,
            DataDeOriginiacao,
            DataDaProximaParcela,
            DataDeVencimento,
            DataDeRenegociacao,
            CPF,
            CodigoDoProduto,
            CodigoDoContrato,
            DiasEmAtraso,
            QuantidadeDeParcelasAbertas,
            ValorSaldo,
            ValorPDD,
            ValorOperacao,
            RendaDoCliente,
            ValorDaProximaParcela,
            RatingContabilFim) 
            VALUES (%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,
                    %s,%s)"""
sql.adicione_multiplos_registros(query, pdd_registros)

In [50]:
# os dados dessas querys nao vem ordenados
parametros = (datetime.datetime(2018,1,1), datetime.datetime(2018,2,1))
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeOriginiacao BETWEEN %s AND %s
                      ''',parametros)